In [1]:

import pandas as pd
tr_data = pd.read_csv('train.csv')
ts_data=pd.read_csv('test.csv')


In [2]:

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

from imblearn.datasets import fetch_datasets
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.ensemble import RUSBoostClassifier

import matplotlib.pyplot as plt 


ModuleNotFoundError: No module named 'xgboost'

In [ ]:
tr_data.head()

In [ ]:
tr_data.columns

# counting the mising values

In [ ]:
tr_data.isna().sum()

In [ ]:
ts_data.isnull().sum()

In [3]:
#because backers_count is not available in test_data , option is either to drop or use train data to predict for test data and furtherr build final model
#currently dropping 

In [4]:
tr_data.drop(['backers_count'],axis=1,inplace=True)

In [5]:
ts_data.shape

(63465, 12)

In [6]:
len(set(tr_data['name']))

107839

In [7]:
tr_data.shape

(108129, 13)

In [8]:
len(set(tr_data['keywords']))

108129

In [9]:
tr_data.tail()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,final_status
108124,kkst542772509,Custom Wearable Art,Sacred geometry art and custom clothing. Embro...,2000.0,custom-wearable-art,False,US,USD,1433095651,1433095654,1429382307,1430503651,0
108125,kkst615785942,Painted Modern Wall art Mid-century! $100 for...,I want to create pieces very similar to this o...,15000.0,zacharys-art-project,False,US,USD,1433096040,1433096041,1428339909,1428344094,0
108126,kkst1421442073,Creating High Quality Art Prints from the Arti...,I am seeking to produce prints to promote my r...,320.0,creating-high-quality-art-prints-from-the-arti...,False,US,USD,1433096334,1433096337,1430420564,1430504334,1
108127,kkst2022543055,back to basics,A self sustaining homestead with organic produ...,35000.0,back-to-basics-2,False,US,USD,1433096580,1433096584,1430072594,1430421727,0
108128,kkst169301870,"Happy Olive, Mediterranean Wraps","Mediterranean Wraps, done right!",25000.0,happy-olive-mediterranean-wraps,False,US,USD,1433096938,1433096940,1429144432,1429208938,0


# converting dates from linux format to general date time format 

In [10]:
from datetime import datetime

In [11]:
def get_date(x):
  return(datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))

In [12]:
tr_data['deadline']=tr_data['deadline'].apply(lambda x:get_date(x))

In [13]:

tr_data['state_changed_at']=tr_data['state_changed_at'].apply(lambda x:get_date(x))

tr_data['created_at']=tr_data['created_at'].apply(lambda x:get_date(x))

tr_data['launched_at']=tr_data['launched_at'].apply(lambda x:get_date(x))

In [14]:
tr_data.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,final_status
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them...,20.0,drawing-for-dollars,False,US,USD,2009-05-03,2009-05-03,2009-04-24,2009-04-24,1
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,False,US,USD,2009-05-15,2009-05-16,2009-04-28,2009-04-29,0
2,kkst183622197,Mr. Squiggles,So I saw darkpony's successfully funded drawin...,30.0,mr-squiggles,False,US,USD,2009-05-22,2009-05-22,2009-05-12,2009-05-12,0
3,kkst597742710,Help me write my second novel.,Do your part to help out starving artists and ...,500.0,help-me-write-my-second-novel,False,US,USD,2009-05-29,2009-05-29,2009-04-29,2009-04-29,1
4,kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, current...",2000.0,support-casting-my-sculpture-in-bronze,False,US,USD,2009-05-31,2009-05-31,2009-05-01,2009-05-01,0


In [15]:
ts_data['deadline']=ts_data['deadline'].apply(lambda x:get_date(x))

ts_data['state_changed_at']=ts_data['state_changed_at'].apply(lambda x:get_date(x))

ts_data['created_at']=ts_data['created_at'].apply(lambda x:get_date(x))

ts_data['launched_at']=ts_data['launched_at'].apply(lambda x:get_date(x))

In [16]:
ts_data.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at
0,kkst917493670,Bràthair.,"My first film, of many to come. Trying to purs...",7000.0,brathair,False,US,USD,2015-12-08,2015-12-08,2015-10-28,2015-10-29
1,kkst1664901914,THE SCREENWRITER,A young man that has earned his master's in sc...,35000.0,the-screenwriter,False,US,USD,2016-01-22,2016-01-22,2015-12-16,2015-12-18
2,kkst925125077,The Hornets Nest the Fairmont Heights Story,Film about a high school constructed for negro...,49500.0,the-hornets-nest-the-fairmont-heights-story,False,US,USD,2016-01-03,2016-01-03,2015-11-26,2015-11-28
3,kkst1427645275,BROTHERS Season 2 - Groundbreaking Transgender...,The acclaimed series about a group of transgen...,40000.0,brothers-season-2-groundbreaking-transgender-male,False,US,USD,2015-10-16,2015-10-16,2015-08-30,2015-09-16
4,kkst1714249266,Blackdom the movie,Blackdom's history offers a new narrative tha...,20000.0,blackdom-the-movie,False,US,USD,2016-05-01,2016-05-01,2016-02-18,2016-03-18


# creating features using dates 

In [17]:
tr_data.dtypes

project_id                object
name                      object
desc                      object
goal                     float64
keywords                  object
disable_communication       bool
country                   object
currency                  object
deadline                  object
state_changed_at          object
created_at                object
launched_at               object
final_status               int64
dtype: object

In [18]:
date_columns=['deadline','launched_at','created_at','state_changed_at']

In [19]:
for cols in date_columns:
    tr_data[cols]=pd.to_datetime(tr_data[cols])
    ts_data[cols]=pd.to_datetime(ts_data[cols])

In [20]:
tr_data['diff_launch_dead']=tr_data['deadline']-tr_data['launched_at']

tr_data['diff_created_dead']=tr_data['deadline']-tr_data['created_at']

In [21]:
ts_data['diff_launch_dead']=ts_data['deadline']-ts_data['launched_at']

ts_data['diff_created_dead']=ts_data['deadline']-ts_data['created_at']

In [22]:
tr_data.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,final_status,diff_launch_dead,diff_created_dead
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them...,20.0,drawing-for-dollars,False,US,USD,2009-05-03,2009-05-03,2009-04-24,2009-04-24,1,9 days,9 days
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,False,US,USD,2009-05-15,2009-05-16,2009-04-28,2009-04-29,0,16 days,17 days
2,kkst183622197,Mr. Squiggles,So I saw darkpony's successfully funded drawin...,30.0,mr-squiggles,False,US,USD,2009-05-22,2009-05-22,2009-05-12,2009-05-12,0,10 days,10 days
3,kkst597742710,Help me write my second novel.,Do your part to help out starving artists and ...,500.0,help-me-write-my-second-novel,False,US,USD,2009-05-29,2009-05-29,2009-04-29,2009-04-29,1,30 days,30 days
4,kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, current...",2000.0,support-casting-my-sculpture-in-bronze,False,US,USD,2009-05-31,2009-05-31,2009-05-01,2009-05-01,0,30 days,30 days


In [23]:
ts_data.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,diff_launch_dead,diff_created_dead
0,kkst917493670,Bràthair.,"My first film, of many to come. Trying to purs...",7000.0,brathair,False,US,USD,2015-12-08,2015-12-08,2015-10-28,2015-10-29,40 days,41 days
1,kkst1664901914,THE SCREENWRITER,A young man that has earned his master's in sc...,35000.0,the-screenwriter,False,US,USD,2016-01-22,2016-01-22,2015-12-16,2015-12-18,35 days,37 days
2,kkst925125077,The Hornets Nest the Fairmont Heights Story,Film about a high school constructed for negro...,49500.0,the-hornets-nest-the-fairmont-heights-story,False,US,USD,2016-01-03,2016-01-03,2015-11-26,2015-11-28,36 days,38 days
3,kkst1427645275,BROTHERS Season 2 - Groundbreaking Transgender...,The acclaimed series about a group of transgen...,40000.0,brothers-season-2-groundbreaking-transgender-male,False,US,USD,2015-10-16,2015-10-16,2015-08-30,2015-09-16,30 days,47 days
4,kkst1714249266,Blackdom the movie,Blackdom's history offers a new narrative tha...,20000.0,blackdom-the-movie,False,US,USD,2016-05-01,2016-05-01,2016-02-18,2016-03-18,44 days,73 days


# create year ,month and date as feature

In [24]:
tr_data['created_month']=tr_data['created_at'].dt.month

tr_data['created_year']=tr_data['created_at'].dt.year

tr_data['created_day']=tr_data['created_at'].dt.day

In [25]:
tr_data['launched_at_month']=tr_data['launched_at'].dt.month
tr_data['launched_at_year']=tr_data['launched_at'].dt.year
tr_data['launched_at_day']=tr_data['launched_at'].dt.day

In [26]:
tr_data.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,...,launched_at,final_status,diff_launch_dead,diff_created_dead,created_month,created_year,created_day,launched_at_month,launched_at_year,launched_at_day
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them...,20.0,drawing-for-dollars,False,US,USD,2009-05-03,2009-05-03,...,2009-04-24,1,9 days,9 days,4,2009,24,4,2009,24
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,False,US,USD,2009-05-15,2009-05-16,...,2009-04-29,0,16 days,17 days,4,2009,28,4,2009,29
2,kkst183622197,Mr. Squiggles,So I saw darkpony's successfully funded drawin...,30.0,mr-squiggles,False,US,USD,2009-05-22,2009-05-22,...,2009-05-12,0,10 days,10 days,5,2009,12,5,2009,12
3,kkst597742710,Help me write my second novel.,Do your part to help out starving artists and ...,500.0,help-me-write-my-second-novel,False,US,USD,2009-05-29,2009-05-29,...,2009-04-29,1,30 days,30 days,4,2009,29,4,2009,29
4,kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, current...",2000.0,support-casting-my-sculpture-in-bronze,False,US,USD,2009-05-31,2009-05-31,...,2009-05-01,0,30 days,30 days,5,2009,1,5,2009,1


In [27]:
ts_data['created_month']=ts_data['created_at'].dt.month

ts_data['created_year']=ts_data['created_at'].dt.year

ts_data['created_day']=ts_data['created_at'].dt.day

ts_data['launched_at_month']=ts_data['launched_at'].dt.month

ts_data['launched_at_year']=ts_data['launched_at'].dt.year

ts_data['launched_at_day']=ts_data['launched_at'].dt.day

In [28]:
ts_data.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,diff_launch_dead,diff_created_dead,created_month,created_year,created_day,launched_at_month,launched_at_year,launched_at_day
0,kkst917493670,Bràthair.,"My first film, of many to come. Trying to purs...",7000.0,brathair,False,US,USD,2015-12-08,2015-12-08,2015-10-28,2015-10-29,40 days,41 days,10,2015,28,10,2015,29
1,kkst1664901914,THE SCREENWRITER,A young man that has earned his master's in sc...,35000.0,the-screenwriter,False,US,USD,2016-01-22,2016-01-22,2015-12-16,2015-12-18,35 days,37 days,12,2015,16,12,2015,18
2,kkst925125077,The Hornets Nest the Fairmont Heights Story,Film about a high school constructed for negro...,49500.0,the-hornets-nest-the-fairmont-heights-story,False,US,USD,2016-01-03,2016-01-03,2015-11-26,2015-11-28,36 days,38 days,11,2015,26,11,2015,28
3,kkst1427645275,BROTHERS Season 2 - Groundbreaking Transgender...,The acclaimed series about a group of transgen...,40000.0,brothers-season-2-groundbreaking-transgender-male,False,US,USD,2015-10-16,2015-10-16,2015-08-30,2015-09-16,30 days,47 days,8,2015,30,9,2015,16
4,kkst1714249266,Blackdom the movie,Blackdom's history offers a new narrative tha...,20000.0,blackdom-the-movie,False,US,USD,2016-05-01,2016-05-01,2016-02-18,2016-03-18,44 days,73 days,2,2016,18,3,2016,18


# using label encoder for the string based features

In [29]:
le=LabelEncoder()

NameError: name 'LabelEncoder' is not defined

In [30]:
d={False:0,True:1}
tr_data['disable_communication'].replace(d,inplace=True)

In [31]:
ts_data['disable_communication'].replace(d,inplace=True)

In [32]:
object_cols=[ 'country', 'currency']

In [33]:
tr_data['country'].value_counts()

US    92033
GB     8758
CA     3736
AU     1880
NL      705
NZ      355
SE      240
DK      196
NO      114
IE      111
DE        1
Name: country, dtype: int64

In [34]:
#how add unknown tag
for c in object_cols:
    print (c)
    le.fit(tr_data[c])
    ts_data[c] = ts_data[c].map(lambda s: '<unknown>' if s not in le.classes_ else s)
    le.classes_ = np.append(le.classes_, '<unknown>')
    tr_data[c] = le.transform(tr_data[c])
    ts_data[c] = le.transform(ts_data[c])

country


NameError: name 'le' is not defined

In [35]:
tr_data.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,...,launched_at,final_status,diff_launch_dead,diff_created_dead,created_month,created_year,created_day,launched_at_month,launched_at_year,launched_at_day
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them...,20.0,drawing-for-dollars,0,US,USD,2009-05-03,2009-05-03,...,2009-04-24,1,9 days,9 days,4,2009,24,4,2009,24
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,0,US,USD,2009-05-15,2009-05-16,...,2009-04-29,0,16 days,17 days,4,2009,28,4,2009,29
2,kkst183622197,Mr. Squiggles,So I saw darkpony's successfully funded drawin...,30.0,mr-squiggles,0,US,USD,2009-05-22,2009-05-22,...,2009-05-12,0,10 days,10 days,5,2009,12,5,2009,12
3,kkst597742710,Help me write my second novel.,Do your part to help out starving artists and ...,500.0,help-me-write-my-second-novel,0,US,USD,2009-05-29,2009-05-29,...,2009-04-29,1,30 days,30 days,4,2009,29,4,2009,29
4,kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, current...",2000.0,support-casting-my-sculpture-in-bronze,0,US,USD,2009-05-31,2009-05-31,...,2009-05-01,0,30 days,30 days,5,2009,1,5,2009,1


In [36]:
ts_data.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,diff_launch_dead,diff_created_dead,created_month,created_year,created_day,launched_at_month,launched_at_year,launched_at_day
0,kkst917493670,Bràthair.,"My first film, of many to come. Trying to purs...",7000.0,brathair,0,US,USD,2015-12-08,2015-12-08,2015-10-28,2015-10-29,40 days,41 days,10,2015,28,10,2015,29
1,kkst1664901914,THE SCREENWRITER,A young man that has earned his master's in sc...,35000.0,the-screenwriter,0,US,USD,2016-01-22,2016-01-22,2015-12-16,2015-12-18,35 days,37 days,12,2015,16,12,2015,18
2,kkst925125077,The Hornets Nest the Fairmont Heights Story,Film about a high school constructed for negro...,49500.0,the-hornets-nest-the-fairmont-heights-story,0,US,USD,2016-01-03,2016-01-03,2015-11-26,2015-11-28,36 days,38 days,11,2015,26,11,2015,28
3,kkst1427645275,BROTHERS Season 2 - Groundbreaking Transgender...,The acclaimed series about a group of transgen...,40000.0,brothers-season-2-groundbreaking-transgender-male,0,US,USD,2015-10-16,2015-10-16,2015-08-30,2015-09-16,30 days,47 days,8,2015,30,9,2015,16
4,kkst1714249266,Blackdom the movie,Blackdom's history offers a new narrative tha...,20000.0,blackdom-the-movie,0,US,USD,2016-05-01,2016-05-01,2016-02-18,2016-03-18,44 days,73 days,2,2016,18,3,2016,18


In [37]:
tr_cols=list(tr_data.columns)
ts_cols=list(ts_data.columns)

In [38]:
rm_cols=['project_id','name','desc','keywords','launched_at','created_at','deadline','state_changed_at','diff_launch_dead',
 'diff_created_dead']

In [39]:
required_cols_tr_data=[]
required_cols_ts_data=[]

In [40]:
for c in tr_cols:
    if c not in rm_cols:
        required_cols_tr_data.append(c)
for x in ts_cols:
    if x not in rm_cols:
        required_cols_ts_data.append(x)

In [41]:
required_cols_tr_data  

['goal',
 'disable_communication',
 'country',
 'currency',
 'final_status',
 'created_month',
 'created_year',
 'created_day',
 'launched_at_month',
 'launched_at_year',
 'launched_at_day']

In [42]:
required_cols_ts_data

['goal',
 'disable_communication',
 'country',
 'currency',
 'created_month',
 'created_year',
 'created_day',
 'launched_at_month',
 'launched_at_year',
 'launched_at_day']

# Splitting of data

In [43]:
y=tr_data['final_status']


In [48]:
X=tr_data[required_cols_tr_data]

In [49]:
X.drop(['final_status'],axis=1,inplace=True)

In [50]:
X.head()

,goal,disable_communication,country,currency,created_month,created_year,created_day,launched_at_month,launched_at_year,launched_at_day
0,20.0,0,US,USD,4,2009,24,4,2009,24
1,300.0,0,US,USD,4,2009,28,4,2009,29
2,30.0,0,US,USD,5,2009,12,5,2009,12
3,500.0,0,US,USD,4,2009,29,4,2009,29
4,2000.0,0,US,USD,5,2009,1,5,2009,1


In [51]:

X_train, X_test, y_train, y_test =train_test_split(X,y,test_size=0.10)

NameError: name 'train_test_split' is not defined

In [52]:
X_train.shape

NameError: name 'X_train' is not defined

In [53]:
#fiiting the tree classifier and then an ensmebled claassifier

In [54]:
#tree classifier

In [55]:
clf = DecisionTreeClassifier()

In [55]:
#why this error ,drop the date fatures because we already have the time stamp features as date ,year ,month 

In [56]:
clf.fit(X_train,y_train)

NameError: name 'clf' is not defined

In [57]:
y_pred=clf.predict(X_test)

NameError: name 'clf' is not defined

In [58]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='weighted') 

NameError: name 'y_test' is not defined

In [ ]:
#draw the confusing matrix

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
#fit the boosted model, tune the hyperparams , build confusion matrix ,using over /undersampling 

In [ ]:
from xgboost import XGBClassifier

In [ ]:
#tune scle_pos_weight for considering the imbalanced classes 

In [ ]:
xgb=XGBClassifier(max_depth=7,n_estimators=300,scale_pos_weight=1.4)

In [ ]:
model=xgb.fit(X_train,y_train)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
f1_score(y_test, y_pred, average='weighted') 

In [ ]:
confusion_matrix(y_test, y_pred)

In [59]:
#tune the model to improve 

In [60]:
final_prediction=model.predict(ts_data[required_cols_ts_data])

NameError: name 'model' is not defined

In [61]:
final_prediction

NameError: name 'final_prediction' is not defined

In [62]:
dataframe=pd.DataFrame(ts_data['project_id'])

In [65]:
dataframe['predicted_status']=final_prediction

NameError: name 'final_prediction' is not defined

In [66]:
dataframe.to_csv('cloudzies_prediction_file.csv')
